# Headline analysis

In [ ]:
import os
import matplotlib.pyplot as pl
import numpy as np
import pandas as pd
import xarray as xr
import dask
from tqdm.auto import tqdm

In [ ]:
os.makedirs('../plots', exist_ok=True)

In [ ]:
pl.rcParams['figure.figsize'] = (8.7/2.54, 8.7/2.54)
pl.rcParams['font.size'] = 7
pl.rcParams['font.family'] = 'Arial'
pl.rcParams['ytick.direction'] = 'in'
pl.rcParams['ytick.minor.visible'] = True
pl.rcParams['ytick.major.right'] = True
pl.rcParams['ytick.right'] = True
pl.rcParams['xtick.direction'] = 'in'
pl.rcParams['xtick.minor.visible'] = True
pl.rcParams['xtick.major.top'] = True
pl.rcParams['xtick.top'] = True
pl.rcParams['axes.spines.top'] = True
pl.rcParams['axes.spines.bottom'] = True
pl.rcParams['figure.dpi'] = 150

In [ ]:
# ds = xr.open_dataset("../results/MESSAGEix-GLOBIOM_GE_SSP2_int_lc_15.nc")
def add_modelscen_coord(ds1):
    filename = ds1.encoding["source"]
    return ds1.assign(model_scen=filename.split('/')[-1][:-3])

ds = xr.open_mfdataset(str("../results/*.nc"), parallel=True, preprocess=add_modelscen_coord, concat_dim='model_scen', combine="nested")

In [ ]:
ds

In [ ]:
ds.coords["scenario"]

In [ ]:
ds.temperature

In [ ]:
scen_index = {
    'MFR_tech_MCLE20': 3,
    'MFR_tech_MC20_100': 1,
    'MFR_tech_MCGT100': 2, 
    'MFR_explore': 4, 
    'MFR_fossil': 6,
    'MFR_fair': 5,
    'MFR_behavior': 7,
    'MFR_develop': 8
}

In [ ]:
colors = {
    'MFR_tech_MCLE20': '#0b0626',
    'MFR_tech_MC20_100': '#31173f',
    'MFR_tech_MCGT100': '#5f1f51', 
    'MFR_explore': '#8f2858', 
    'MFR_fossil': '#bb3854',
    'MFR_fair': '#df5446',
    'MFR_behavior': '#f77a30',
    'MFR_develop': '#ffa600'
}

In [ ]:
for scenario, index in scen_index.items():
    pl.plot(
        ds.year[265:], 
        np.median(
            ds.temperature[:, 265:, index, :] - ds.temperature[:, 265:, 0, :],
            axis=(0,2)
        ),
        label=scenario,
        color=colors[scenario]
    );
pl.legend()
pl.title("Median across climate and parent scenario")
pl.ylabel('°C relative to CLE_Baseline');
pl.xlim(2015, 2100);

In [ ]:
notremind = np.where(ds.model_scen=="REMIND 2.1_R2p1_SSP5-NPi", -999, np.arange(113))

In [ ]:
notremind[notremind>=0]

In [ ]:
agg = np.ones((112, 87, 8, 1001)) * np.nan
for ind in range(8):
    agg[:, :, ind, :] = ds.temperature[notremind[notremind>=0], 265:, ind+1, :] - ds.temperature[notremind[notremind>=0], 265:, 0, :]

In [ ]:
pl.fill_between(
    ds.year[265:], 
    np.percentile(
        agg,
        5,
        axis=(0,2,3)
    ), np.percentile(
        agg,
        95,
        axis=(0,2,3)
    ),
    color='0.8',
#    alpha=0.3,
    lw=0
)
for scenario, index in scen_index.items():
    
#     pl.fill_between(
#         ds.year[265:], 
#         np.percentile(
#             ds.temperature[, 265:, index, :] - ds.temperature[:, 265:, 0, :],
#             5,
#             axis=(0,2)
#         ), np.percentile(
#             ds.temperature[:, 265:, index, :] - ds.temperature[:, 265:, 0, :],
#             95,
#             axis=(0,2)
#         ),
#         color=colors[scenario],
#         alpha=0.3,
#         lw=0
#     )
    pl.plot(
        ds.year[265:], 
        np.median(
            ds.temperature[notremind[notremind>0], 265:, index, :] - ds.temperature[notremind[notremind>0], 265:, 0, :],
            axis=(0,2)
        ),
        label=scenario,
        color=colors[scenario]
    );
pl.legend()
pl.title("Avoided warming: climate and scenario uncertainty")
pl.ylabel('°C relative to CLE_Baseline');
pl.xlim(2015, 2100);
pl.tight_layout()
pl.savefig('../plots/warming_climate_scenario_uncertainty.png')
pl.savefig('../plots/warming_climate_scenario_uncertainty.pdf')

In [ ]:
for scenario, index in scen_index.items():
    pl.fill_between(
        ds.year[265:], 
        np.percentile(
            np.median(ds.temperature[:, 265:, index, :] - ds.temperature[:, 265:, 0, :], axis=0),
            5,
            axis=1
        ), np.percentile(
            np.median(ds.temperature[:, 265:, index, :] - ds.temperature[:, 265:, 0, :], axis=0),
            95,
            axis=1
        ),
        color=colors[scenario],
        alpha=0.3,
        lw=0
    )
    pl.plot(
        ds.year[265:], 
        np.median(
            np.median(ds.temperature[:, 265:, index, :] - ds.temperature[:, 265:, 0, :], axis=0),
            axis=1
        ),
        label=scenario,
        color=colors[scenario]
    );
pl.legend()
pl.title("Avoided warming: climate uncertainty")
pl.ylabel('°C relative to CLE_Baseline');
pl.xlim(2015, 2100);
pl.tight_layout()
pl.savefig('../plots/warming_climate_uncertainty.png')
pl.savefig('../plots/warming_climate_uncertainty.pdf')

In [ ]:
for scenario, index in scen_index.items():
    pl.fill_between(
        ds.year[265:], 
        np.percentile(
            np.median(ds.temperature[:, 265:, index, :] - ds.temperature[:, 265:, 0, :], axis=2),
            5,
            axis=0
        ), np.percentile(
            np.median(ds.temperature[:, 265:, index, :] - ds.temperature[:, 265:, 0, :], axis=2),
            95,
            axis=0
        ),
        color=colors[scenario],
        alpha=0.3,
        lw=0
    )
    pl.plot(
        ds.year[265:], 
        np.median(
            np.median(ds.temperature[:, 265:, index, :] - ds.temperature[:, 265:, 0, :], axis=2),
            axis=0
        ),
        label=scenario,
        color=colors[scenario]
    );
pl.legend()
pl.title("Avoided warming: scenario uncertainty")
pl.ylabel('°C relative to CLE_Baseline');
pl.xlim(2015, 2100);
pl.tight_layout()
pl.savefig('../plots/warming_scenario_uncertainty.png')
pl.savefig('../plots/warming_scenario_uncertainty.pdf')

In [ ]:
for scenario, index in scen_index.items():
    pl.plot(
        ds.year[265:], 
        np.median(
            ds.ch4_concentration[:, 265:, index, :] - ds.ch4_concentration[:, 265:, 0, :],
            axis=(0,2)
        ),
        label=scenario,
        color=colors[scenario]
    );
pl.legend()
pl.title("Median across climate and parent scenario")
pl.ylabel('ppb difference relative to CLE_Baseline');
pl.xlim(2015, 2100);
pl.tight_layout()
pl.savefig('../plots/concentration_climate_scenario_median.png')
pl.savefig('../plots/concentration_climate_scenario_median.pdf')

In [ ]:
df_emissions = pd.read_csv('../data/gains_scenarios_harmonized.csv')
df_emissions

In [ ]:
df_emissions[['scenario', 'variant']] = df_emissions['scenario'].str.split("|", expand=True)

In [ ]:
df_emissions['model_scen'] = df_emissions['model'] + '_' + df_emissions['scenario']

In [ ]:
df_emissions

In [ ]:
len(df_emissions['model_scen'].unique())

In [ ]:
ds.model_scen

In [ ]:
for ims, model_scen in tqdm(enumerate(df_emissions['model_scen'].unique()), total=113):
    for scenario, index in scen_index.items():
        med_t = np.median(
            ds.temperature[ims, 266:, index, :] - ds.temperature[ims, 266:, 0, :],
            axis=1
        )
        #print(med_t)
        if np.min(med_t) < -1:
            print(model_scen, scenario)
        pl.plot(
            np.cumsum(
                df_emissions.loc[
                    (df_emissions['model_scen']==model_scen) & 
                    (df_emissions['variant']==scenario) &
                    (df_emissions['variable']=='Emissions|CH4'),
                    '2015':'2100'
                ].values - df_emissions.loc[
                    (df_emissions['model_scen']==model_scen) & 
                    (df_emissions['variant']=='Baseline_CLE') &
                    (df_emissions['variable']=='Emissions|CH4'),
                    '2015':'2100'
                ].values
            ),
 
            med_t,
            
            color=colors[scenario]
        )

pl.xlabel('Cumulative reduction emissions compared to Baseline, MtCH4')
pl.ylabel('Avoided warming, °C')
pl.tight_layout()
pl.savefig('../plots/warming_cumulativeCH4.png')
pl.savefig('../plots/warming_cumulativeCH4.pdf')

In [ ]:
pl.plot(np.arange(1750, 2102), ds.temperature.loc[dict(model_scen='REMIND 2.1_R2p1_SSP5-NPi', scenario='Baseline_CLE')]);

In [ ]:
pl.plot(np.arange(1750, 2102), ds.ch4_concentration.loc[dict(model_scen='REMIND 2.1_R2p1_SSP5-NPi', scenario='Baseline_CLE')]);

In [ ]:
ds.close()